## CSV and Excel Parsing - Structured Data

In [4]:
import pandas as pd
import os

In [5]:
os.makedirs("data/structured_files", exist_ok=True)

In [7]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

df = pd.DataFrame(data)
df.to_csv('data/structured_files/products.csv', index=False)

In [9]:
# Save as Excel with multiple sheets

with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)

    # Add another sheet
    summary_data = {
        'Category': ['Electronics', "Accessories"],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)

## CSV Processing


In [10]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

In [15]:
print(' CSV Loader - Row-based Documents')

csv_loader = CSVLoader(
    file_path="data/structured_files/products.csv",
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"'
    }
)

csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\n First Document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

print(csv_docs)

 CSV Loader - Row-based Documents
Loaded 5 documents (one per row)

 First Document:
Content: Product: Laptop
Category: Electronics
Price: 999.99
Stock: 50
Description: High-performance laptop with 16GB RAM and 512GB SSD
Metadata: {'source': 'data/structured_files/products.csv', 'row': 0}
[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_f

## Excel Processing

In [24]:
# Method 1: Using Pandas for full control
from langchain_core.documents import Document
from typing import List
import pandas as pd

print(" Pandas-based Excel Processing")

def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""

    # Read all Sheets

    excel_file = pd.ExcelFile(filepath)

    documents = []

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)

        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name} \n"
        sheet_content += f"Columns: {', '.join(df.columns)} \n"
        sheet_content += f"Rows: {len(df)} \n\n"
        sheet_content += df.to_string(index=False)

        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )

        documents.append(doc)

    return documents

 Pandas-based Excel Processing


In [26]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

excel_docs

Processed 2 sheets


[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'excel_sheet'}, page_content='Sheet: Products \nColumns: Product, Category, Price, Stock, Description \nRows: 5 \n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary \nColumns: Category, Total